In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
#from sklearn.pipeline import Pipeline
#from pandas_profiling import ProfileReport
#from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

In [ ]:
!pip uninstall pandas-profiling -y
!pip install pandas-profiling==3.2.0
!pip uninstall markupsafe -y
!pip install markupsafe==2.0.1

In [ ]:
# load data
D_Train = pd.read_csv(r"../content/train.csv")
D_test = pd.read_csv(r"../content/test.csv")
D_Train.replace('?', np.NaN, inplace=True)
D_test.replace('?', np.NaN, inplace=True)
D_Train = D_Train.astype(float)
D_test = D_test.astype(float)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,7,8,10,11,15,19,23,26,30,36,37,39,49,52,56,58,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,7,8,10,11,15,19,23,26,30,36,37,39,49,52,58,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
Num_col1 = D_Train.drop(columns=['forecasting period','id','class'])
Num_col = list(Num_col1.columns)
Cate_col = ['forecasting period']
Drop_Col = ['id']

In [ ]:
Y = D_Train['class']
X = D_Train.round(decimals = 8)
X = X.drop(columns='class')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,shuffle=True, stratify=Y)

In [ ]:
def outlier_removal(X,factor):
    X = pd.DataFrame(X).copy()
    for i in range(X.shape[1]):
        x = pd.Series(X.iloc[:,i]).copy()
        q1 = x.quantile(0.05)
        q3 = x.quantile(0.95)
        iqr = q3 - q1
        lower_bound = q1 - (factor * iqr)
        upper_bound = q3 + (factor * iqr)
        X.iloc[((X.iloc[:,i] < lower_bound) | (X.iloc[:,i] > upper_bound)),i] = np.nan 
    return X

#creating outlier_remover object using FunctionTransformer with factor=1.5
outlier_remover = FunctionTransformer(outlier_removal,kw_args={'factor':1.7})

In [ ]:
def outlier_replace(Data):
  Y = pd.DataFrame(Data).copy()
  for i in range(Y.shape[1]):
    y = pd.Series(Y.iloc[:,i]).copy()
    q1 = y.quantile(0.05)
    q3 = y.quantile(0.95)
    iqr1 = q3 - q1
    lower_bound = q1 - (1.5 * iqr1)
    upper_bound = q3 + (1.5 * iqr1)
    Y.iloc[(Y.iloc[:,i] < lower_bound),i] =  lower_bound
    Y.iloc[(Y.iloc[:,i] > upper_bound),i] = upper_bound
  return Y
outlier_replacer = FunctionTransformer(outlier_replace)

In [ ]:
numeric_transformer = Pipeline(steps=[
                                     ('outlier_replacer', outlier_replacer),
                                     ('meanimputer', SimpleImputer(strategy='median',add_indicator=True)),
                                     ('stdscaler', MinMaxScaler())
                                     ])
categorical_transformer = Pipeline(steps=[
                                         ('Replaceimputer', SimpleImputer(strategy='constant',fill_value=0)),
                                         ('onehotenc', OneHotEncoder(handle_unknown='ignore'))
                                         ])

col_transformer = ColumnTransformer(transformers=[('drop_columns', 'drop', Drop_Col),
                                                ('numeric_processing',numeric_transformer, Num_col),
                                                ('categorical_processing', categorical_transformer, Cate_col)],
                                                remainder='drop')
model = MLPClassifier()
pipeline1 = Pipeline([('transform_column', col_transformer),('model',model)])
pipeline1.fit(X_train,y_train)

#transformed_train = pipeline.transform(X_train)
pipeline = Pipeline([('transform_column', col_transformer)])
pipeline.transform(X_test)
pipeline.transform(D_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


array([[0.50481755, 0.48971767, 0.39855814, ..., 0.        , 0.        ,
        1.        ],
       [0.36543055, 0.52395035, 0.50766716, ..., 1.        , 0.        ,
        0.        ],
       [0.81799107, 0.40986777, 0.58088518, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.52352306, 0.45407798, 0.47552994, ..., 0.        , 0.        ,
        0.        ],
       [0.47587393, 0.49719469, 0.46914391, ..., 0.        , 1.        ,
        0.        ],
       [0.50696329, 0.43956763, 0.53528656, ..., 0.        , 1.        ,
        0.        ]])

In [ ]:
models = [MLPClassifier(activation='relu',solver='adam',shuffle=True)]
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
metric = make_scorer(f1_score)
scores = cross_val_score(pipeline1, X_train, y_train, scoring=metric, cv=cv, n_jobs=-1)

KeyboardInterrupt: ignored

In [ ]:
model_names = ['MLP']
# Draw bboxplot 
plt.figure(figsize=(16, 8))
plt.boxplot(scores)
plt.xlabel('Model', fontsize=16)
plt.ylabel('Score', fontsize=16)
plt.xticks(np.arange(len(model_names))+1, model_names, rotation=45, fontsize=16)
plt.title("Scores Metrics", fontsize=18)

NameError: ignored

<Figure size 1152x576 with 0 Axes>

In [ ]:
resulting = pipeline1.predict(D_test)
df1 = pd.DataFrame(resulting)
df1.to_excel("../content/ketquade.xlsx",sheet_name='Result')

Việc sử dụng các phương pháp balance data không làm mô hình tốt hơn, chưa hiểu.
